In [1]:
from arcgis.gis import GIS
gis = GIS('<ORGANIZATION>','<USERNAME>','<PASSWORD>')
import pandas as pd
import datetime
import numpy as np
from arcgis.features import FeatureLayer

/opt/conda/lib/python3.7/site-packages/arcgis/gis/__init__.py:575: UserWarning:

You are logged on as smandanas_dangermond with an administrator role, proceed with caution.



## Generate list of EagleWatch users, import table, filter table to today's entries only

In [3]:
#import hosted table with user information
intake_url = '<User info table service URL>'
intake = FeatureLayer(intake_url,gis)
intake = intake.query(where='OBJECTID<10000')
intake_table = intake.sdf

In [4]:
#create dictionaries to identify users based on emails or AGOL usernames
emails = dict(zip(intake_table.email_address,intake_table.first_name+' '+intake_table.last_name))
usernames = dict(zip(intake_table.UserAGOL_1,intake_table.first_name+' '+intake_table.last_name))

In [5]:
#import table to be updated
data_url = '<Table to update service URL>'
layer = FeatureLayer(data_url)
#layer.properties.fields

In [7]:
#add table as dataframe
query = layer.query(where='OBJECTID<1000000')
dataframe = query.sdf

In [6]:
#Create the time window to go back and update names
#This notebook runs once daily, so the time window is one day
now_UTC = datetime.date.today()
now = now_UTC - datetime.timedelta(days = 1)

In [8]:
#filter the table to only include entries from the past day
dataframe['CreationDate_2'] = pd.to_datetime(dataframe['CreationDate_2']).dt.date
dataframe = dataframe[dataframe['CreationDate_2'] >= now]

In [9]:
len(dataframe)

119

## Update the Data

In [11]:
#remove any spaces present in the email field
#create a temporary Survey_By column and 
dataframe['vol_email'] = dataframe['vol_email'].str.replace(' ','')
dataframe['Survey_By'] = np.where(dataframe['Creator_2']!='vol_eaglewatch',dataframe['Creator_2'].map(usernames),dataframe['vol_email'].map(emails))

## Format and publish data back to AGOL

In [12]:
#Replace official 'SurvBy' column with the temporary column for corrected data 'Survey_By'
dataframe['SurvBy'] = dataframe['Survey_By']

In [13]:
#Delete all columns from the table that we don't need to update the data
dataframe.drop(columns=['NestID', 'SubstrateTy', 'Disturbance', 'Occupied',
       'Active', 'Successful', 'Monitored', 'VolHours', 'Notes',
       'vol_email', 'AdultsSeen', 'GlobalID', 'CreationDate', 'Creator',
       'EditDate', 'Editor', 'Season', 'VisitDate', 'Approved', 'noteSurv',
       'emailAGOL', 'userAGOL', 'Moved', 'Status', 'subs_change', 'A_banded',
       'band_color', 'A_behave', 'EagletBehav', 'EagletsSeen', 'Fledged',
       'Perished', 'CreationDate_1', 'Creator_1', 'EditDate_1', 'Editor_1',
       'CreationDate_2', 'Creator_2', 'EditDate_2', 'Editor_2', 'Verification',
       'Survey_By'],inplace=True)

In [14]:
#Convert dataframe to featureset
fs = dataframe.spatial.to_featureset().value

In [15]:
#break updates into chunks to reduce possibility of breaking
def chunk_it(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

chunk_size = 50
chunks = chunk_it(fs['features'],chunk_size)

In [16]:
#update each chunk
count = 1
for chunk in chunks:
    try:
        layer.edit_features(updates=chunk)
        print(count)
        #count += 1
    except Exception as e:
        raise ValueError(f'error adding chunk ::{e}')

1
1
1
